In [1]:
import pandas as pd
import os

In [2]:
# --- INPUT FILE ---
# The full log file from your V40 run.
V40_LOG_FILE = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V40_20251025_193828/V40_full_inference_log.csv"

# --- OUTPUT FILES ---
# This file will contain all "relevant" images that S1 incorrectly filtered out.
OUTPUT_S1_FAILURES = "S1_failures_to_review.csv"

# This file will contain the specific S2 confusion pairs we want to fix.
OUTPUT_S2_FAILURES = "S2_failures_to_review.csv"

# --- CLASS DEFINITIONS ---
# All classes that are NOT 'hard_case'
RELEVANT_CLASSES = [
    'anger', 'contempt', 'disgust', 'fear', 'happiness',
    'neutral', 'questioning', 'sadness', 'surprise',
    'neutral_speech', 'speech_action'
]

# The specific S2 confusions we want to target
DISGUST_CONFUSIONS = ['anger', 'sadness', 'fear']

print("Configuration loaded.")

Configuration loaded.


In [3]:
def run_triage():
    if not os.path.exists(V40_LOG_FILE):
        print(f"❌ ERROR: Cannot find the log file: {V40_LOG_FILE}")
        return

    print(f"Loading log file: {V40_LOG_FILE}...")
    df = pd.read_csv(V40_LOG_FILE)
    print(f"Loaded {len(df)} total records.")
    
    # --- 1. Find S1 Failures (False Negatives) ---
    # S1 failures are images where the true_label is relevant,
    # but the top1_label is empty (NaN) because S1 discarded it.
    
    s1_failure_mask = (
        df['true_label'].isin(RELEVANT_CLASSES) & 
        df['top1_label'].isnull()
    )
    s1_failures_df = df[s1_failure_mask].copy()
    
    if not s1_failures_df.empty:
        print(f"\n[S1 Triage] Found {len(s1_failures_df)} relevant images that S1 discarded.")
        s1_failures_df.to_csv(OUTPUT_S1_FAILURES, index=False)
        print(f"   -> Saved to '{OUTPUT_S1_FAILURES}'")
    else:
        print("\n[S1 Triage] ✅ No S1 false negatives found.")

    
    # --- 2. Find S2 Failures (Confusion Pairs) ---
    # We are looking for specific misclassifications from the V40 log.
    
    # Condition 1: Contempt <-> Questioning
    contempt_v_questioning = (
        (df['true_label'] == 'contempt') & (df['top1_label'] == 'questioning')
    )
    questioning_v_contempt = (
        (df['true_label'] == 'questioning') & (df['top1_label'] == 'contempt')
    )
    
    # Condition 2: Disgust -> [anger, sadness, fear]
    disgust_confusions = (
        (df['true_label'] == 'disgust') & (df['top1_label'].isin(DISGUST_CONFUSIONS))
    )
    
    s2_failure_mask = (
        contempt_v_questioning | 
        questioning_v_contempt | 
        disgust_confusions
    )
    s2_failures_df = df[s2_failure_mask].copy()

    if not s2_failures_df.empty:
        print(f"\n[S2 Triage] Found {len(s2_failures_df)} specific S2 confusion pairs.")
        # Sort by the true label to make review easier
        s2_failures_df = s2_failures_df.sort_values(by='true_label')
        s2_failures_df.to_csv(OUTPUT_S2_FAILURES, index=False)
        print(f"   -> Saved to '{OUTPUT_S2_FAILURES}'")
    else:
        print("\n[S2 Triage] ✅ No specific S2 confusion pairs found.")
        
    print("\n triage complete.")

# Run the function
run_triage()

Loading log file: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V40_20251025_193828/V40_full_inference_log.csv...
Loaded 25895 total records.

[S1 Triage] Found 2337 relevant images that S1 discarded.
   -> Saved to 'S1_failures_to_review.csv'

[S2 Triage] Found 1 specific S2 confusion pairs.
   -> Saved to 'S2_failures_to_review.csv'

 triage complete.
